=====DATSCIW261 ASSIGNMENT #7=====


MIDS UC Berkeley, Machine Learning at Scale

DATSCIW261 ASSIGNMENT #7

#### Hetal Chandaria (hetalchandaria@berkeley.edu) 

W261 - 2 , ASSIGNMENT #7

Submission Date : 

Group : 4

### ===HW 9.0: Short answer questions===

What is PageRank and what is it used for in the context of web search?

What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to 
compute the steady stade distibuton?

OPTIONAL: In topic-specific pagerank, how can we insure that the irreducible property is satified? (HINT: see HW9.4)

<h2><span style="color: blue"> Answer </span></h2>
<li>
<span style="color: blue"> What is PageRank and what is it used for in the context of web search? </span> </li><li>
<span style="color: blue"> What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the steady stade distibuton?</span></li><li>
<span style="color: blue"> OPTIONAL: In topic-specific pagerank, how can we insure that the irreducible property is satified? (HINT: see HW9.4)</span></li>
</li>





### ===HW 9.1: MRJob implementation of basic PageRank===

Write a basic MRJob implementation of the iterative PageRank algorithm
that takes sparse adjacency lists as input (as explored in HW 7).
Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), 
and further, distributes the mass of dangling nodes with each iteration
so that the output of each iteration is correctly normalized (sums to 1).
[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page,
chooses the next page to which it will move by clicking at random, with probability d,
one of the hyperlinks in the current page. This probability is represented by a so-called
‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer
jumps to any web page in the network. If a page is a dangling end, meaning it has no
outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform
distribution and “teleports” to that page]


As you build your code, use the test data

s3://ucb-mids-mls-networks/PageRank-test.txt
Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck
your work with the true result, displayed in the first image
in the Wikipedia article:

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

A,0.033

B,0.384

C,0.343

D,0.039

E,0.081

F,0.039

G,0.016

H,0.016

I,0.016

J,0.016

K,0.016



<h2><span style="color: blue"> Answer </span></h2>

Implementation approach:

1. First we will do data analysis to determine the number of nodes
2. Second we will use the number of nodes as input to the pageRank job
3. Page Rank implementation will have 2 Map Reduce jobs : 
<ul>
    <li>One to compute the page ranks. Look at the psuedo code in #5 below </li>
    <li>Second one to handle dangling nodes. Look at the formula in #6 below</li> 
   </ul>

4.  Teleportation parameter $\alpha$ is set to 0.15 

5. We are going to reference the below algorithm for implementing the first MR job for page rank.

<img src = "./img/pagerank_algo.png"> </img>

6 . Formula to take into account the random jump factor and dangling nodes
<ul>
<li>Dangling nodes are nodes in the graph that have no outgoing edges, i.e., their adjacency lists are empty. </li>
<li>
The proper treatment of PageRank mass “lost” at the dangling nodes is to re- distribute it across all nodes in the graph evenly. </li>
<li>
At the same time, we can take into account the random jump factor. For each node, its current PageRank value p is updated to the final PageRank value p′ according to the following formula:

$p' = \alpha \frac{1}{N} + (1=\alpha)(\frac{m}{N}+p)$

where,

$\alpha$ = teleportation / random jump factor

m = is the missing node mass,

N = total number of nodes

p = probability of node



In [243]:
%%writefile graph_eda.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import get_jobconf_value
from ast import literal_eval
import sys

class graph_eda(MRJob):
    
    empty_dict={}
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer=self.reducer)
               ]
        
    def mapper(self,_,line):
        t = line.strip().split('\t')
        node = t[0]
        neighbors = literal_eval(t[1])
        yield node,neighbors
        for neighbor in neighbors:
            yield neighbor, self.empty_dict
            
    def reducer(self,key,values):
        valList = [val for val in values]
        tot = len(valList)
        neighbor = None
        for i in range(tot):
            if not valList[i]:
                continue
            neighbor = (valList[i])
        if neighbor==None:
            yield key, self.empty_dict
        else: yield key,neighbor
        
        
        
if __name__ == '__main__':
    graph_eda.run()    


Overwriting graph_eda.py


In [247]:
!python graph_eda.py PageRank-test.txt -q > PageRank-test.txt.out

In [254]:
%%writefile mrjob_pageRank.py

from __future__ import division
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import get_jobconf_value
from ast import literal_eval
import sys

class pageRank(MRJob):
    dangling_mass = 0
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer=self.reducer)
            ,MRStep(mapper_init=self.mapper_teleporation_init,mapper=self.mapper_dangling)
               ]
    
    def configure_options(self):
        super(pageRank, self).configure_options()
        self.add_passthrough_option(
            '--nodes',dest='N', type='int', default="1",
            help='Nodes: Total number of nodes in the graph')
        self.add_passthrough_option(
            '--teleportation',dest='alpha', type='float', default="0.15",
            help='Teleportation: Random jump factor to handle dangling nodes')
        
    def mapper(self,_,line):
#         Output from this mapper is of below format
#             "A"	"graph|{}"
#             "dangling"	0.09090909090909091
#             "B"	"graph|{'C': 1}"
#             "C"	0.09090909090909091
#             "C"	"graph|{'B': 1}"
#             "B"	0.09090909090909091
#             "D"	"graph|{'A': 1, 'B': 1}"
#             "A"	0.045454545454545456
#             "B"	0.045454545454545456
    
#         line = line.replace("\"","'")
      
        t = line.strip().split('\t')
        node = t[0]
        tmp_dict = literal_eval(t[1])
        neighbors= None
        if '*rank' in tmp_dict.keys():
            pr = float(tmp_dict['*rank'])
            del tmp_dict['*rank']
            neighbors = tmp_dict
        else :
            pr = 1/self.options.N # this is our first run and probability for each node is equal
            neighbors = tmp_dict
        yield node.replace("\"",""), ('{0}|{1}'.format('graph',neighbors))
#         print neighbors
        out_links =  len(neighbors)
        if (out_links != 0):
            for neighbor in neighbors:
                yield neighbor , pr/out_links
        else : yield 'dangling',pr
    
        
            
            
    def reducer(self,key,value):
        '''
        Output from this reducer is of the below form
        "A"	"{}|0.0454545454545"
        "B"	"{'C': 1}|0.348484848485"
        "C"	"{'B': 1}|0.0909090909091"
        '''
        node = key
        valList = [val for val in value]
        graph_struct=None
        pr = 0
        if node == 'dangling': # code for summing up dangling mass . We wil write it to file 
            self.dangling_mass = sum(valList)
            with open('dangling_mass.txt', 'w') as f:
                f.write('{0}'.format(self.dangling_mass))
            f.close()

        else: # code for summing up probability and yielding data out 
            for val in valList:
                if type(val)== str:
                    tmp = val.split('|')
                    graph_struct = literal_eval(tmp[1])
                else :
                    pr += val
            graph_struct['*rank'] = pr
            yield node,graph_struct
            
    def mapper_teleporation_init(self):
        self.dangling_mass = 0
        try: #using try catch block here because if there are no dangling nodes than the 'dangling.txt' will not exist
            with open('dangling_mass.txt','r') as f:
                l = f.readlines()
                self.dangling_mass = float(l[0])
            open(dangling_mass.txt, 'w').close()
        except Exception as e: 
            pass
#         print "From second map reduce job",self.dangling_mass
    
    def mapper_dangling(self,key,value):
        node = key.replace("\"","")
#         node = node.replace("\\","")
#         tmp = value.split('|')
        graph_struct = literal_eval(str(value))
        pr = float(value['*rank'])
        new_pr = 0
        if(self.dangling_mass > 0):
            new_pr = (self.options.alpha/self.options.N)+(1-self.options.alpha)*((self.dangling_mass/self.options.N)+pr)
        else: new_pr = pr
        graph_struct['*rank'] = new_pr
        yield node,graph_struct
if __name__ == '__main__':
    pageRank.run()    

Overwriting mrjob_pageRank.py


In [257]:
#driver for page rank so we can iterate in loop 

%reload_ext autoreload
%autoreload 2
from mrjob_pageRank import pageRank
from ast import literal_eval
import pprint

!python graph_eda.py PageRank-test.txt -q > PageRank-test.txt.out

input_file = 'PageRank-test.txt.out'

iter_num = 0
MAX_ITERATIONS = 20
num_nodes = 11
teleporation = 0.15

prev_ranks = []
curr_ranks = []
sum_partial_diff_PR = float('inf')

# After so many iterations (TBD: Convergence Criteria)
while sum_partial_diff_PR > .0001:
    print "\n\nIteration: " ,iter_num
    mr_job = pageRank(args=[input_file,
                            '--nodes','11',
                            '--teleportation',teleporation,
                            '--file', 'dangling_mass.txt',
                            '--no-strict-protocol'
                           ])
    
    with mr_job.make_runner() as runner: 
        runner.run()
        
        if iter_num == 0:
            prev_ranks = [0]*num_nodes
        else:
            prev_ranks = curr_ranks[:]
            del curr_ranks[:]
        
        total_page_rank = 0
        ranks = []
       
        with open(input_file, 'w') as f:
            for line in runner.stream_output():
                f.write(line)
                node,parsed_line = mr_job.parse_output_line(line)
#                 print parsed_line
                tmp = literal_eval(str(parsed_line))
                curr_ranks.append(round(float(tmp['*rank']),3))
                total_page_rank += float(tmp['*rank'])

                        
        sum_partial_diff_PR = sum([abs(x-y) for x,y in zip(curr_ranks, prev_ranks)])
        
        print "\nPage Rank (Sum): {1}".format(iter_num , total_page_rank)
        print "Difference in Ranks: {1}".format(iter_num, sum_partial_diff_PR)
        print 'Page Ranks in Current Iteration: ', curr_ranks
    
    iter_num  += 1



Iteration:  0

Page Rank (Sum): 1.0
Difference in Ranks: 1.001
Page Ranks in Current Iteration:  [0.059, 0.317, 0.098, 0.046, 0.33, 0.046, 0.021, 0.021, 0.021, 0.021, 0.021]


Iteration:  1

Page Rank (Sum): 1.0
Difference in Ranks: 0.645
Page Ranks in Current Iteration:  [0.038, 0.261, 0.288, 0.112, 0.099, 0.112, 0.018, 0.018, 0.018, 0.018, 0.018]


Iteration:  2

Page Rank (Sum): 1.0
Difference in Ranks: 0.38
Page Ranks in Current Iteration:  [0.064, 0.407, 0.238, 0.045, 0.118, 0.045, 0.017, 0.017, 0.017, 0.017, 0.017]


Iteration:  3

Page Rank (Sum): 1.0
Difference in Ranks: 0.301
Page Ranks in Current Iteration:  [0.038, 0.314, 0.365, 0.052, 0.087, 0.052, 0.019, 0.019, 0.019, 0.019, 0.019]


Iteration:  4

Page Rank (Sum): 1.0
Difference in Ranks: 0.227
Page Ranks in Current Iteration:  [0.039, 0.419, 0.283, 0.041, 0.094, 0.041, 0.017, 0.017, 0.017, 0.017, 0.017]


Iteration:  5

Page Rank (Sum): 1.0
Difference in Ranks: 0.189
Page Ranks in Current Iteration:  [0.034, 0.34, 0.37

### ===HW 9.2: Exploring PageRank teleportation and network plots===
In order to overcome  problems such as disconnected components, the damping factor (a typical value for d is 0.85) can be varied. 
Using the graph in HW1, plot the test graph (using networkx, https://networkx.github.io/) for several values of the damping parameter alpha,
so that each nodes radius is proportional to its PageRank score. In particular you should
do this for the following damping factors: [0,0.25,0.5,0.75, 0.85, 1]. Note your plots should look like the following:

https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.svg

In [273]:
import networkx as nx
import matplotlib.pyplot as plt
from mrjob_pageRank import pageRank
from ast import literal_eval
import pprint

!python graph_eda.py PageRank-test.txt -q > PageRank-test.txt.out

input_file = 'PageRank-test.txt.out'

num_nodes = 11
damping= [0,0.25,0.5,0.75, 0.85, 1]

# for d in damping:
print " Damping factor :",d
mr_job = pageRank(args=[input_file,
                        '--nodes','11',
                        '--teleportation',0.15,
                        '--file', 'dangling_mass.txt',
                        '--no-strict-protocol'
                       ])
G=nx.Graph()
neighbor_dict={}
node_pr ={}
with mr_job.make_runner() as runner: 
    runner.run()

    for line in runner.stream_output():
        node,parsed_line = mr_job.parse_output_line(line)
        tmp = literal_eval(str(parsed_line))
        node_pr[node]=tmp['*rank']
        del tmp['*rank']
        neighbors = tmp.keys()
        neighbor_dict[node] = neighbors
        G.add_node(node)
                  
for node, edges in neighbor_dict.iteritems():
    if (len(edges) > 0):
        for e in edges:
            G.add_edge(node,e)


 Damping factor : 1


C B
B C
E B
E D
E F
D A
D B
G B
G E
F B
F E
I B
I E
H B
H E
K E
J E


### ===HW 9.3: Applying PageRank to the Wikipedia hyperlinks network===

Run your PageRank implementation on the Wikipedia dataset for 10 iterations,
and display the top 100 ranked nodes (with alpha = 0.85).

Run your PageRank implementation on the Wikipedia dataset for 50 iterations,
and display the top 100 ranked nodes (with teleportation factor of 0.15). 
Have the top 100 ranked pages changed? Comment on your findings. Plot the pagerank values for the top 100 pages resulting from the 50 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  




### ===HW 9.4: Topic-specific PageRank implementation using MRJob===

Modify your PageRank implementation to produce a topic specific PageRank implementation,
as described in:

http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf

Note in this article that there is a special caveat to ensure that the transition matrix is irreducible.
This caveat lies in footnote 3 on page 3:

	A minor caveat: to ensure that M is irreducible when p
	contains any 0 entries, nodes not reachable from nonzero
	nodes in p should be removed. In practice this is not problematic.

and must be adhered to for convergence to be guaranteed.

Run topic specific PageRank on the following randomly generated network of 100 nodes:

s3://ucb-mids-mls-networks/randNet.txt (also available on Dropbox)

which are organized into ten topics, as described in the file:

s3://ucb-mids-mls-networks/randNet_topics.txt  (also available on Dropbox)

Since there are 10 topics, your result should be 11 PageRank vectors
(one for the vanilla PageRank implementation in 9.1, and one for each topic
with the topic specific implementation). Print out the top ten ranking nodes 
and their topics for each of the 11 versions, and comment on your result. 
Assume a teleportation factor of 0.15 in all your analyses.

One final and important comment here:  please consider the 
requirements for irreducibility with topic-specific PageRank.
In particular, the literature ensures irreducibility by requiring that
nodes not reachable from in-topic nodes be removed from the network.

This is not a small task, especially as it it must be performed
separately for each of the (10) topics.

So, instead of using this method for irreducibility, 
please comment on why the literature's method is difficult to implement,
and what what extra computation it will require.
Then for your code, please use the alternative, 
non-uniform damping vector:

vji = beta*(1/|Tj|); if node i lies in topic Tj

vji = (1-beta)*(1/(N - |Tj|)); if node i lies outside of topic Tj

for beta in (0,1) close to 1. 

With this approach, you will not have to delete any nodes.
If beta > 0.5, PageRank is topic-sensitive, 
and if beta < 0.5, the PageRank is anti-topic-sensitive. 
For any value of beta irreducibility should hold,
so please try beta=0.99, and perhaps some other values locally,
on the smaller networks.

### ===HW 9.5: Applying topic-specific PageRank to Wikipedia===

Here you will apply your topic-specific PageRank implementation to Wikipedia,
defining topics (very arbitrarily) for each page by the length (number of characters) of the name of the article mod 10,
so that there are 10 topics. Once again, print out the top ten ranking nodes 
and their topics for each of the 11 versions, and comment on your result.
Assume a teleportation factor of 0.15 in all your analyses.



### HW 9.6: TextRank (OPTIONAL)

What is TextRank. Describe the main steps in the algorithm. Why does TextRank work?
Implement TextRank in MrJob for keyword phrases (not just unigrams) extraction using co-occurrence based similarity measure with with sizes of N = 2 and 3. And evaluate your code using the following example using precision, recall, and FBeta (Beta=1):

"Compatibility of systems of linear constraints over the set of natural numbers
Criteria of compatibility of a system of linear Diophantine equations, strict 
inequations, and nonstrict inequations are considered. Upper bounds for
components of a minimal set of solutions and algorithms of construction of 
minimal generating sets of solutions for all types of systems are given. 
These criteria and the corresponding algorithms for constructing a minimal 
supporting set of solutions can be used in solving all the considered types of 
systems and systems of mixed types." 

The extracted keywords should in the following set:

linear constraints, linear diophantine equations, natural numbers, non-strict inequations, strict inequations, upper bounds

